# ディープラーニングのホットドッグ検出器のレシピ

研究開発部の画像解析の担当のレシェックです。techlife を書くのは初めてです。よろしくお願いいたします。


最先端の機械学習を使うためには、いつも自分のスキルアップが必要です。そのために、毎日論文を読んだり、新しいオープンソースのコードを試してみたり、クックパッドのデータで実験しています。これはちょっと料理の練習と似ています。時々新しいモデルを学習させるのは料理をオーブンに入れるのと同じ気持ちです。オーブンの温度とか、学習率と同じで、低すぎだとよく焼けず、高すぎだと焦げてしまいます。しかし、ちゃんと他のリサーチャーの論文やブログの中のレシピを見ながら、自分のデータでモデルを学習させると失敗せずに済むかもしれません。

この開発者ブログでは、クックパッドの機械学習のレシピを紹介したいと思います。

![hot dog highlight](images/picnic-993906_640-processed.jpg)

このブログで使っているテスト画像は[Pixabay](http://pixabay.com)から取得した、Creative Commonsのライセンスの写真です。

## 説明

クックパッドは[料理/非料理のモデル](http://techlife.cookpad.com/entry/2017/09/14/161756)を開発しています。ここでは、このモデルのミニチュア版のレシピを紹介します。カテゴリは「料理」と「非料理」の代わりに、「ホットドッグ」と「非ホットドッグ」にします。そして、パッチ化した画像に対する認識モデルを使って、画像の中でホットドッグがどこにあるかを検出します。

## 調理器具

 - python
 - Keras
 - numpy
 - pillow (PIL)

[Keras](https://keras.io/ja/)はTensorflow、CNTKやTheano上で動く高水準のライブラリーです。Keras は特に画像データに対して、単なる学習以外にも前処理などでも様々な機能があります。

## 材料

Kaggleから、[Hot Dog - Not Hot Dog](https://www.kaggle.com/dansbecker/hot-dog-not-hot-dog/data)のデーターセットをダウンロードしてください。Kaggleに登録が必要です。

ダウンロードした後、`seefood.zip`を`unzip`してください。

アーカイブの中に、2つのディレクトリ：`train`と`test`があります：
```
seefood/train/not_hot_dog
seefood/train/hot_dog
seefood/test/not_hot_dog
seefood/test/hot_dog
```
`hot_dog`ディレクトリの中にホットドッグの画像が入っており、`not_hot_dog`の中にそれ以外の画像が入っています。新しい機械学習のレシピを開発する時は、テストデータを分けた方がいいですが、今回の場合は画像が少ないので、テストデータも学習に使いましょう。
```
mkdir seefood/all
cp -r seefood/test/* seefood/train/* seefood/all
```
以降では、`seefood/all`のディレクトリを使います。

## データ拡張

Keras のモバイルネットは（224px・224px）のフィックスサイズの画像しか認識できないので、これから学習や認識用にサイズを変換します。

In [1]:
import tensorflowjs as tfjs

/Users/leszek-rybicki/.pyenv/versions/3.6.0/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
IMG_SIZE=[224, 224]

テストデータを学習に使っても、このデータセットはまだ小さいので、データ拡張を使いましょう。

Kerasの`ImageDataGenerator`は学習時に、画像を一つずつ変換します。

In [3]:
import keras.preprocessing.image

image_generator = keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        shear_range=0.0,
        width_shift_range=0.1,
        height_shift_range=0.1,
        rotation_range=10,
        fill_mode="wrap",
        vertical_flip=True,
        horizontal_flip=True
)

上の`image_generator`を`"seefood/all"`のディレクトリで動かします。

In [4]:
train_generator = image_generator.flow_from_directory(
    "seefood/all",
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode="categorical",
    classes=["not_hot_dog", "hot_dog"]
)

Found 998 images belonging to 2 classes.


## モデルの作り方


以下のレシピでは、3 個のモデルを 3 層のスポンジケーキのように積み重ねています。

1. `base_model`はMobileNetです。転移学習のために使います。
2. その上の、`patch_model`は画像のパッチごとに分類できます。
3. さらにその上の`classifier`は「ホットドッグ」や「非ホットドッグ」の二値分類器です。

まず`keras`を`import`します：

In [5]:
import keras

ベースとして、Googleで開発されたMobileNetというモデルを使います。

`weights="imagenet"`は、ILSVRCのコンペティションのデータセットで学習されたパラメタを使って、転移学習することを意味しています

In [6]:
base_model = keras.applications.mobilenet.MobileNet(
    input_shape=IMG_SIZE + [3], 
    weights="imagenet",
    include_top=False
)

ベースモデルの一番上のフィーチャサイズは1024です。パッチレイヤが学習できるようにちょっと下げましょう。

In [7]:
drop1 =base_model.output
conv_filter = keras.layers.convolutional.Conv2D(
    4, (1,1),
    activation="relu",
    use_bias=True,
    kernel_regularizer=keras.regularizers.l2(0.001)
)(drop1)

パッチレイヤもConv2Dのタイプのレイヤです。この場合、`softmax`を使えば、パッチごとに分類できるようになります。

In [8]:
drop2 = conv_filter
patch = keras.layers.convolutional.Conv2D(
    2, (3, 3),
    name="patch",
    activation="softmax",
    use_bias=True,
    padding="same",
    kernel_regularizer=keras.regularizers.l2(0.001)
)(drop2)

これでパッチモデルができました。

In [9]:
patch_model = keras.models.Model(
    inputs=base_model.input, 
    outputs=patch
)

パッチモデルをベースにして、最後の出力レイヤを追加して分類モデルを作ります。

In [10]:
pool = keras.layers.GlobalAveragePooling2D()(patch)
logits = keras.layers.Activation("softmax")(pool)

In [11]:
classifier = keras.models.Model(
    inputs=base_model.input, 
    outputs=logits
)

## 学習

ベースモデルは学習させません。

In [12]:
for layer in base_model.layers:
    layer.trainable = False

そしで、全体のモデルを`compile`します：

In [13]:
classifier.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])

では、学習を始めましょう！

いくつか実験をした結果、以下のようにnot_hot_dogのクラスのclass_weightを高くするほうが良いことが分かりました。

In [14]:
%%time
classifier.fit_generator(
    train_generator, 
    class_weight={0: .75, 1: .25}, 
    epochs=10
)

Epoch 1/10
32/32 [==============================] - 136s 4s/step - loss: 0.2953 - acc: 0.4949
Epoch 2/10
32/32 [==============================] - 258s 8s/step - loss: 0.2671 - acc: 0.5977
Epoch 3/10
32/32 [==============================] - 139s 4s/step - loss: 0.2490 - acc: 0.6996
Epoch 4/10
32/32 [==============================] - 121s 4s/step - loss: 0.2351 - acc: 0.7668
Epoch 5/10
32/32 [==============================] - 114s 4s/step - loss: 0.2393 - acc: 0.7498
Epoch 6/10
32/32 [==============================] - 139s 4s/step - loss: 0.2370 - acc: 0.7467
Epoch 7/10
32/32 [==============================] - 153s 5s/step - loss: 0.2314 - acc: 0.7801
Epoch 8/10
32/32 [==============================] - 148s 5s/step - loss: 0.2371 - acc: 0.7553
Epoch 9/10
32/32 [==============================] - 146s 5s/step - loss: 0.2274 - acc: 0.8024
Epoch 10/10
32/32 [==============================] - 136s 4s/step - loss: 0.2277 - acc: 0.8014
CPU times: user 1h 6min 57s, sys: 2min 10s, total: 1h 9min 

In [15]:
tfjs.converters.save_keras_model(patch_model, './model/')

このデータセットの場合、１０エポックぐらいが良さそうです。パッチベースを使っているので、精度は１００％にならないほうがいいです。７０％ぐらいがちょうどいいです。

私の MacBook Pro では１０エポックで２０分ぐらいかかりました。

## チェック

画像とデータの変換のために、PILとnumpyを使います。

In [16]:
import numpy as np
from PIL import Image

画像をインファレンスする前に、`numpy`のデータに変換します。

In [17]:
def patch_infer(img):
    data = np.array(img.resize(IMG_SIZE))/255.0
    patches = patch_model.predict(data[np.newaxis])
    return patches

そして、元の画像とインファレンス結果をビジュアライズします：

In [18]:
def overlay(img, patches, threshold=0.99):
    # transposeはパッチをクラスごとに分けます。
    patches = patches[0].transpose(2, 0, 1)
    # hot_dogパッチ - not_hot_dogパッチ
    patches = patches[1] - patches[0]
    # 微妙なパッチをなくして
    patches = np.clip(patches, threshold, 1.0)
    patches = 255.0 * (patches - threshold) / (1.0 - threshold)
    # 数字を画像にして
    patches = Image.fromarray(patches.astype(np.uint8)).resize(img.size, Image.BICUBIC)
    # もとの画像を白黒に
    grayscale = img.convert("L").convert("RGB").point(lambda p: p * 0.5)
    # パッチをマスクに使って、元の画像と白黒の画像をあわせて
    composite = Image.composite(img, grayscale, patches)
    return composite

まとめて、インファレンスとビジュアライズを一つのファンクションにすると：

In [19]:
def process_image(path, border=8):
    img = Image.open(path)
    patches = patch_infer(img)
    result = overlay(img, patches)
    # 元の画像と変換された画像をカンバスに並べます
    canvas = Image.new(
        mode="RGB", 
        size=(img.width * 2 + border, img.height), 
        color="white")
    canvas.paste(img, (0,0))
    canvas.paste(result, (img.width + border, 0))
    return canvas

In [20]:
import os
for filename in os.listdir("images"):
    path = os.path.join("images", filename)
    if not path.endswith("640.jpg"):
        continue
    canvas = process_image(path)
    canvas.save(path.replace('.jpg', '-processed.jpg'))

では、結果を見てみましょう！

![CC0 by NadinLisa](images/barbecue-283889_640-processed.jpg)
きれいですね！

![CC0 by pairswing](images/coffee-2429489_640-processed.jpg)
ホットドッグの色はちょっととなりのコーヒーに移りましたが、ほとんど大丈夫です。

![CC0 by HannahChen](images/hot-dog-657039_640-processed.jpg)
フォーカスが足りないところは認識にならなかったみたいです。なぜでしょう？学習データにフォーカスが当たらないホットドッグがなかったからです。

![CC0 by skeeze](images/picnic-993906_640-processed.jpg)
こちらも、左側のホットドッグはフォーカスが当たっておらず、モデルはホットドッグを認識できませんでした。

ホットドッグではない画像は？
![CC0 by sharonang](images/fish-and-chip-3039746_640-processed.jpg)

![CC0 by dimitrisvetsikas1969](images/cat-3276083_640-processed.jpg)


![CC0 by wildfaces](images/dog-3289600_640-processed.jpg)

![CC0 by Free Photos](images/vw-camper-336606_640-processed.jpg)


ホットドッグではない画像には、パッチはゼロやゼロに近い値になります。

## まとめ

転移学習を使えば、データが少なくても、それなりの識別器が作れますね！

パッチごとの分類を使えば、画像の中の認識したいフィーチャーを可視化できます。

モバイルネット(MobileNet)のおかげで、CPU でもモデルを学習できます。

いかがでしたでしょうか。 クックパッドでは、機械学習を用いて新たなサービスを創り出していける方を募集しています。 興味のある方はぜひ話を聞きに遊びに来て下さい。